For more on Hubble's discovery: <br>
https://www.nasa.gov/content/about-story-edwin-hubble <br>
https://news.uchicago.edu/explainer/hubble-constant-explained

Source:
http://supernova.lbl.gov/Union/, https://arxiv.org/abs/1105.3470, DOI 10.1088/0004-637X/746/1/85

# A Classic Experiment: Edwin Hubble's Discovery of the Expansion of the Universe

In the early 20th century, Edwin Hubble (University of Chicago alumnus) significantly contributed to the understanding of the universe via the observation that a number of stars and galaxies moved away from the Earth with a velocity $v$ that is proportional to their distance $d$ from Earth. This relationship is known as Hubble's law

$v=H_0 d$

in which $H_0$ is the Hubble constant, typically measured in units of km s$^{-1}$ Mpc$^{-1}$ in which Mpc indicates a distance of $10^6$ parsec (1 parsec is about 3.26 light years). The original data used by Hubble can be found in the paper "The Velocity-Distance Relation among Extra-Galactic Nebulae" (Hubble, Edwin; Humason, Milton L., 1931). For some time scientists predicted that the value of the Hubble constant should be around 68 km s$^{-1}$ Mpc$^{-1}$. As more data come into light, it has been suggested that the Hubble constant should be around 70 km s$^{-1}$ Mpc$^{-1}$ with some reports pushing it to 72-74 km s$^{-1}$ Mpc$^{-1}$. Distant galaxies moving away faster than nearby galaxies can serve as an evidence for the expansion of the universe and the Hubble constant indicates the rate at which this expansion is happening (see the blueberry muffin baking analogy in the UChicago New cited above).


We will attempt to fit two models into the Supernova Union data containing distances versus redshift information. One model is a simple linear law and the second one is a nonlinear (extension) of the first model.

In [ ]:
# import libraries for data manipulation
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import math

# Reading csv sample data set into a DataFrame df
label='sn_z_mu_dmu_plow_union2.1.txt'
df = pd.read_csv(label, delimiter='\t', names=["name", "redshift", "distance modulus", "distance modulus uncertainty", "unknown constant"])

# Display how many rows (records) and columns (values per record) we have
print ('Data contains %i rows and %i columns' % (df.shape[0], df.shape[1]))
df.shape

# Display information about our data
df.info()

In [ ]:
# Display the first 5 records from our dataset
df.head()

In [ ]:
# Display the last 5 records from our dataset
df.tail()

In [ ]:
# Let's plot the data
plt.figure()
plt.xlabel('redshift(z)', fontsize=15) #Label x
plt.ylabel('distance modulus', fontsize=15)#Label y
# plot with errorbars
plt.errorbar(df['redshift'],df['distance modulus'],yerr=df['distance modulus uncertainty'],marker='.',linestyle = 'None', color = 'black')
# uncomment the line below to zoom at small redshifts
plt.xlim([0,0.1])
plt.ylim(top=40)
plt.show()

The data is stored in terms of 'distance modulus', $\mu$, (https://en.wikipedia.org/wiki/Distance_modulus) and we need convert it to just 'distance' in parsec. For that, we use the definition

$d = 10^{(\mu/5) + 1}$

and the uncertainty in $\mu$, $\sigma_\mu$ also needs to be propagated so we can obtain the uncertainty in $d$, $\sigma_d$. This can be done using error propagation as

\begin{eqnarray}
\sigma_d &=& \frac{\partial d}{\partial \mu}\, \sigma_\mu \\
 &=& \frac{\log(10)}{5}\, 10^{(\mu/5)+1}\, \sigma_\mu
\end{eqnarray}

In [ ]:
# Let's copy the original dataframe to a new one since we will make column alterations
df_copy = df.copy()

# Let's rename the column to simply 'distance' and make the conversion
df_copy = df_copy.rename(columns={'distance modulus': 'distance', 'distance modulus uncertainty': 'd-uncertainty'})

# Applying the condition using apply and lambda
df_copy['distance'] = 10**(df['distance modulus']/5 + 1) #apply method option: df['distance modulus'].apply(lambda x: 10**(x/5 + 1))
df_copy['d-uncertainty'] = (math.log(10)/5)*df_copy['distance']*df['distance modulus uncertainty']

# We will keep just the rows where z < 0.1
df_reduced = df_copy.loc[ df_copy['redshift'] <= 0.1 ]

In [ ]:
df_reduced.head()

In [ ]:
# Let's replot the new data
plt.figure()
plt.xlabel('redshift(z)', fontsize=15) #Label x
plt.ylabel('distance (parsec)', fontsize=15)#Label y
# plot with errorbars
plt.errorbar(df_reduced['redshift'],df_reduced['distance'],yerr=df_reduced['d-uncertainty'],marker='.',linestyle = 'None', color = 'black')
# uncomment the line below to zoom at small redshifts
plt.xlim([0,0.1])
plt.ylim([0,0.6e9])
plt.show()

In [ ]:
df_reduced.info()

<h2>Linear Curve Fitting</h2


We will apply linear curve fitting method in `lmfit` (https://lmfit.github.io/lmfit-py/index.html). We will perform a linear curve fitting considering that: 
<br>
<ul>
    <li>redshift ($z$) is the predictor/independent variable ($X$);</li>
    <li>distance ($d$) is the response/dependent variable (that we may want to predict)($Y$).</li>
    <li> The data carries incertainty in $Y$.</li>
</ul>

<p>The result of the linear regression procedure is a <b>linear model</b> of the form $Y = b\,X + a$ where</p>

<ul>
    <li>$a$ refers to the <b>intercept</b> of the regression line, in other words: the value of $Y$ when $X$ is 0;</li>
    <li>$b$ refers to the <b>slope</b> of the regression line, in other words: the value with which $Y$ changes when $X$ increases by 1 unit.</li>
</ul>



                         

In [ ]:
# In case you need to install lmfit
#%pip install lmfit

In [ ]:
from lmfit.models import LinearModel

# Linear regression with lmfit
model = LinearModel()
params = model.guess(df_reduced['distance'], x=df_reduced['redshift'])

result = model.fit(df_reduced['distance'], params, x=df_reduced['redshift'], weights=1.0/df_reduced['d-uncertainty'])
print(result.fit_report())

In [ ]:
# plotting the result of lmfit
result.plot_fit()

<b>Constants and conversions required to get $H_0$ in [km s$^{-1}$ Mpc$^{-1}$]<b>
    
Remember that for the linear model, $Y$ is the distance $d$ and $X$ is the redshift $z$. This means that the linear model is written as
    
\begin{equation}
    d = \frac{v}{H_0} + C
\end{equation}
    
with $C$ being any constant and the velocity $v$ is given in terms of the redshift as $v=cz$ ($c$ is the speed of light $\approx 3\times 10^5$ km/s). The distance in our data is in parsec (pc) and we need to convert it to Mega parsec (Mpc), i.e. $d_{pc} = 10^{-6}d_{Mpc}$. Therefore, we can write the relation above as
    
\begin{equation}
    d_{Mpc} = \frac{3\times 10^5 \times 10^6}{H_0} z + C
\end{equation}
    
The linear fiting routine therefore has a slope defined as 
    
\begin{equation}
    \mbox{slope} = \frac{3\times 10^5 \times 10^6}{H_0} 
\end{equation}
    
so, if we wish to extract $H_0$ from the fitted slope, we need to make $H_0 = (3\times 10^5 \times 10^6)/\mbox{slope}$. 
    
The variance in $H_0$ is obtained using error propagation equation as
    
\begin{eqnarray}
    \sigma_{H_0}^2 &=& \left( \frac{\partial H_0}{\partial\, \mbox{slope}} \right )^2\sigma_{\mbox{slope}}^2 \\
        &=& \left( \frac{3\times 10^5 \times 10^6}{\mbox{slope}^2} \right )^2\sigma_{\mbox{slope}}^2
\end{eqnarray}

The standard deviation in $H_0$ is given by $\sigma_{H_0}$ (square root of the variance).

In [ ]:
# Hubble constant taken from the linear fit above done with lmfit
# Getting the parameters from the result object
parameters = result.params

# Hubble constant estimate
H0_lmfit = 1e6*3e5/parameters['slope'].value

# Uncertainty in the Hubble's constant
DH0_lmfit = 1e6*3e5*(parameters['slope'].stderr)/(parameters['slope'].value)**2

# print the values
print("Hubble constant estimated by lmfit (linear model):", H0_lmfit, '+/-', DH0_lmfit)

<b> Extended Hubble law - Nonlinear model </b>

In the cell below, we will implement another curve fitting method using the `curve_fit` method in `scipy.optimize`. For this method to work, we need to create and pass the function we wish to fit. We will use `curve_fit` on a linear and a nonlinear (extended) Hubble's law version which contains a quadratic contribution written as

\begin{equation}
y = \frac{x}{H_0}\left \{1 + \frac{(1-q)}{2} x \right \}
\end{equation}

in which $H_0$ is the Hubble constant and $q$ is a higher-order fitting parameter.

In [ ]:
from scipy.optimize import curve_fit
from scipy import stats

# Fitting with curve_fit of scipy.optimize
def linear_model(x,a,b):
    """
    Linear model for curve fitting.

    Parameters
    ----------
    x : array-like
        The x-values.
    a : float
        The intercept of the line.
    b : float
        The slope of the line.
    
    Returns
    -------
    y : array-like
        The y-values.
    """    
    return a + b*x

def nonlinear(x,h0,q):
    """
    Modified slope expansion model.

    Parameters
    ----------
    x : array-like
        The x-values.
    h0 : float
         Hubble constant.
    q : float
        q-parameter.
    
    Returns
    -------
    y : array-like
        The y-values.
    """    
    val=x*(1e6*3e5/h0)*(1 + ((1-q)*0.5)*x)
    return val

# passing Pandas dataframe to numpy
Y = df_reduced['distance'].to_numpy()
X = df_reduced['redshift'].to_numpy()
dY = df_reduced['d-uncertainty'].to_numpy()

# Fitting with extra argument flags (absolute_sigma)
# We will use absolute_sigma = True since we are assuming that the errors are well-defined.

# Fitting of the linear model
poptlin, pcovlin, infodict, mesg, ier = curve_fit(linear_model, X, Y, sigma=dY, absolute_sigma=True, full_output=True)
a, b = poptlin
h0 = 1e6*3e5/b #Note 1e6 is from pc to Mpc and 3e5 is c in km/s
sigma_a, sigma_b = np.sqrt(np.diag(pcovlin))
sigma_h0 = 1e6*3e5*sigma_b/b**2

# Print the results
print("Linear Model parameters")
print("----------------")
print(f"a = {h0:.3f} +/- {sigma_h0:.3f}")
print(f"b = {b:.3f} +/- {sigma_b:.3f}")
print("\n")

# Fitting of the nonlinear model
poptnl, pcovnl, infodict, mesg, ier = curve_fit(nonlinear, X, Y, sigma=dY, absolute_sigma=True, full_output=True)
# Extract the best-fit parameters and their uncertainties with extra argument flags
h0, q = poptnl
sigma_h0, sigma_q = np.sqrt(np.diag(pcovnl))

# Print the results
print("Nonlinear Model parameters")
print("----------------")
print(f"h0 = {h0:.3f} +/- {sigma_h0:.3f}")
print(f"q = {q:.3f} +/- {sigma_q:.3f}")
print("\n")

# Note on `absolute_sigma` flag in `curve_fit`

In the `scipy.optimize.curve_fit` function, the parameter `absolute_sigma` controls how the uncertainty (standard deviation) in the data is treated during the fitting process.

If `absolute_sigma=False` (default), the uncertainties provided via the sigma parameter are interpreted as relative weights. This means the uncertainties are normalized by the residual sum of squares of the fit, which results in scaling the covariance matrix by a factor dependent on the goodness of fit.

If `absolute_sigma=True`, the uncertainties provided in sigma are treated as absolute standard deviations, meaning that the values in sigma are taken at face value and directly influence the weighting in the least-squares optimization. The covariance matrix returned from `curve_fit` will not be scaled by the residuals in this case.

Key Points:
Use `absolute_sigma=True` if the uncertainties in your data points (given by sigma) are well-established, and you want the fit to respect those exact uncertainties.
Use `absolute_sigma=False` (default) if the uncertainties are relative or approximate, and you want the fitting process to account for the fit quality when estimating the covariance.

In [ ]:
# Let's replot the reduced data with the fitted model of curve_fit
plt.figure()
plt.xlabel('redshift(z)', fontsize=15) #Label x
plt.ylabel('distance (parsec)', fontsize=15)#Label y
# plot of the models
dx = 0.01
xrange = np.arange(min(X),0.1,dx)
plt.plot(xrange, a + b*xrange, label='Linear Model',color='b',zorder=1)
plt.plot(xrange, nonlinear(xrange,h0,q), label='Nonlinear Model',color='r',zorder=1)
# plot data with errorbars
plt.errorbar(df_reduced['redshift'],df_reduced['distance'],yerr=df_reduced['d-uncertainty'],marker='.',label='data',linestyle = 'None', color = 'black',zorder=2)
plt.legend()
plt.show()

In [ ]:
# For the nonlinear fit at this small redshift range, we will obtain the individual residuals and plot them
def plotHist(residuals,x):
    nbins = 30.0
    y0, bin_edges = np.histogram(residuals, bins=int(nbins))
    bin_centers = 0.5*(bin_edges[1:] + bin_edges[:-1])
    norm0=len(residuals)*(bin_edges[-1]-bin_edges[0])/nbins
    
    fig, axs = plt.subplots(2,1, figsize=(6, 6), layout='constrained')
    axs[0].plot(residuals, x, 'k.')
    axs[0].set_ylabel('redshift(z)', fontsize=15)
    axs[0].set_xlabel('residuals', fontsize=15)
    
    axs[1].errorbar(bin_centers,y0/norm0,yerr=y0**0.5/norm0,marker='.',drawstyle = 'steps-mid')
    axs[1].set_xlabel('residuals', fontsize=15)
    axs[1].set_ylabel('Probability', fontsize=15)
    
    
    #for good measure, let's compare this to a gaussian distribution
    k=np.linspace(bin_edges[0],bin_edges[-1],100)
    normal=stats.norm.pdf(k,0,residuals.std())
    axs[1].plot(k,normal,'-',label='Gaussian')
    plt.legend()
    plt.show()

# Calculating residuals
residuals = (df_reduced['distance'] - nonlinear(df_reduced['redshift'],h0,q))/df_reduced['d-uncertainty']
plotHist(residuals,df_reduced['redshift'])


<b>Histogram error bars<b>
    
The error bars in binned (uncorrelated) data can be estimated by considering that the number of entries $h_i$ in each bin $i$ is a random Poisson-distributed with an average value $\mu_i = N p_i$ with $p_i$ being
    
\begin{equation}
    \lim_{N\rightarrow \infty}\, \frac{h_i}{N} = p_i
\end{equation}
    
$N$ being the number of measurements, and the variance $\mbox{Var}[h_i] = \mu_i \approx h_i $. So, the standard deviation is $\sigma_i = \sqrt(h_i)$. This can be normalized by the total number of observations giving $\sqrt(h_i)/N$.
    
<b>Resisual analysis<b>

The plot shows that they are Gaussian distributed, indicating a relatively good fit. Note that the Gaussian plotted in the figure is not a fit, just manually built for comparison.

In [ ]:
# Let's fit the nonlinear model over the whole data
# passing Pandas dataframe to numpy
Y = df_copy['distance'].to_numpy()
X = df_copy['redshift'].to_numpy()
dY = df_copy['d-uncertainty'].to_numpy()

# Fitting with extra argument flags (absolute_sigma)
# We will use absolute_sigma = True since we are assuming that the errors are well-defined.
#popt, pcov, infodict, mesg, ier = curve_fit(linear_model, X, Y, sigma=dY, absolute_sigma=True, full_output=True)
popt, pcov, infodict, mesg, ier = curve_fit(nonlinear, X, Y, sigma=dY, absolute_sigma=True, full_output=True)
# Extract the best-fit parameters and their uncertainties with extra argument flags
h0, q = popt
sigma_h0, sigma_q = np.sqrt(np.diag(pcov))

# Print the results
print("Nonlinear Model parameters")
print("----------------")
print(f"h0 = {h0:.3f} +/- {sigma_h0:.3f}")
print(f"q = {q:.3f} +/- {sigma_q:.3f}")


In [ ]:
# Let's visualize the fit over the whole data
plt.figure()
plt.xlabel('redshift(z)', fontsize=15) #Label x
plt.ylabel('distance (parsec)', fontsize=15)#Label y
dx = 0.01
xrange = np.arange(min(X),max(X),dx)
# plot of the models
plt.plot(xrange, nonlinear(xrange,h0,q), label='Nonlinear Model',color='b',zorder=1)
plt.plot(xrange, linear_model(xrange,a,b), label='Linear Model',color='r',zorder=2)
# plot data with errorbars
plt.errorbar(df_copy['redshift'],df_copy['distance'],yerr=df_copy['d-uncertainty'],marker='.',label='data',linestyle = 'None', color = 'black',zorder=2)
plt.legend()
plt.show()

<b> IMPORTANT:</b> Note that the linear fit above is the linear model that was only fitted to the low range of redshift values! Ideally, we would need to repeat the fitting process with the linear model over the whole data! Nonetheless, we did not do that just to illustrate a case of a "bad fit" and visualize the shift/bias in the residuals later.

<b>$\chi^2$ comparison for goodness of fit<b>

We can have an idea of how good the fit is by comparing the minimum value of $\chi^2$ with the number of degrees of freedom (DoF) given by $n - p$ with $n$ being the number of points in the data and $p$ the number of floating parameters in the model. A reasonably good fit will result in $\chi^2/DoF \approx 1$.

In [ ]:
# Printing the minimum chi^2 value obtained by curve_fit divided by DoF
print(f"SciPy curve_fit \chi^2/DoF: {np.sum(infodict['fvec']**2)/(len(df_copy.index)-2)}")

In [ ]:
# Calculating residuals obtained for the whole dataset (but with the linear model!)
residuals_complete = (df_copy['distance'] - linear_model(df_copy['redshift'],a,b))/df_copy['d-uncertainty']
plotHist(residuals_complete,df_copy['redshift'])

<b>Residual Analysis<b>

The residual plot shows many points tending to a direction. That is indicative that the linear fit does not explain the whole data adequately. Note that the Gaussian drawn in the plot is not a fitting, that is just manually plotted as a guideline to show that the residual distribution is not around zero-mean anymore.